<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/database.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Restore Database from AWS S3</h1>
    </div>
</div>


## Intro

<p class="has-text-justified">
    Introducing a powerful Python notebook designed to simplify performing database restore
</p>

## What you will learn in this notebook:

1. How to restore database from AWS S3 [SQL]


## What benefits do you get out of using the notebook.

1. Using this parameterized notebook, user should be able to perform restore to S3 with just input params.


## Questions?

Reach out to us through our [forum](https://www.singlestore.com/forum).

### Pre-requisites

We will need below parameters to proceed.

<ol type="A">
    <li>Singlestore provides secrets feature to store and manage sensitive data. we will use that to access AWS Access key ID,AWS Secret access key</li>
    <li>Database User should have 'CREATE DATABASE', 'RELOAD' grant</li>
</ol>

<p>Note: </p>
<ol>
    <li>check user grants by running 'show grants'.</li>
    <li>AWS Access key ID,AWS Secret access key of AWS IAM user with S3 read access</li>
</ol>

You may use below query to fetch information about existing backups( filter according to needs)
<code>
    SELECT * from information_schema.MV_BACKUP_HISTORY
</code>

### Imports

In [1]:
import io
import logging

import singlestoredb as s2
from IPython.display import display, HTML

### Variables

In [2]:
restore_database_name = input('Enter database name to restore into')
s3_backup_path = input('Enter S3 Path to backup location')
use_alias = input("Do you like to use different database name. Enter 'Y' for Yes?")
new_database_name = None

if use_alias == 'Y':
    new_database_name = input('Enter new database name')

### Functions to display various alerts

In [3]:
def show_warn(warn_msg):
    """
    Display a warning message in a formatted HTML alert box.

    Parameters
    ----------
    warn_msg : str
        The warning message to display.
    """
    display(HTML(f'''<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>{warn_msg}</p>
    </div>
</div>'''))


def show_error(error_msg):
    """
    Display an error message in a formatted HTML alert box.

    Parameters
    ----------
    error_msg : str
        The error message to display.
    """
    display(HTML(f'''<div class="alert alert-block alert-danger">
    <b class="fa fa-solid fa-exclamation-triangle"></b>
    <div>
        <p><b>Error</b></p>
        <p>{error_msg}</p>
    </div>
</div>'''))


def show_success(success_msg):
    """
    Display a success message in a formatted HTML alert box.

    Parameters
    ----------
    success_msg : str
        The success message to display.
    """
    display(HTML(f'''<div class="alert alert-block alert-success">
    <b class="fa fa-solid fa-check-circle"></b>
    <div>
        <p><b>Success</b></p>
        <p>{success_msg}</p>
    </div>
</div>'''))

### Log Control

In [4]:
def set_logging_enabled(enabled):
    if enabled:
        logging.getLogger().setLevel(logging.INFO)
    else:
        logging.getLogger().setLevel(logging.ERROR)

**Note**

To enable logs
Modify 'set_logging_enabled(False)' to 'set_logging_enabled(True)' in code above

### Fetch Org Secrets

In [5]:
def get_org_secret(lookup_key):
    """
    Get the organization secret using a lookup key.

    Parameters
    ----------
    lookup_key : str
        The lookup key for the organization secret.

    Returns
    -------
    str
        The organization secret.

    """
    return s2.manage_workspaces().org.get_secret(lookup_key).value

### Function to generate restore statement

In [6]:
def get_sql_statement():

    aws_key_id = get_org_secret('BACKUP_APP_AWS_API_KEY_ID')
    aws_secret_key = get_org_secret('BACKUP_APP_AWS_API_SECRET_KEY')
    aws_region = input('BACKUP_APP_AWS_BUCKET_REGION')

    data = io.StringIO()
    data.write('RESTORE DATABASE ' + restore_database_name + ' ')

    if use_alias == 'Y':
        data.write(' AS ' + new_database_name + ' ')

    data.write(' FROM S3 "' + s3_backup_path + '" ')
    data.write(' CONFIG \' {"region":"' + aws_region + '"} \'')
    data.write(' CREDENTIALS \'{"aws_access_key_id":"' + aws_key_id
               + '","aws_secret_access_key":"' + aws_secret_key + '"}\' ')
    logging.debug(f'statement: {data.getvalue()}')
    return data.getvalue()

In [7]:
print('Script execution began')
if connection_url.endswith('/'):
    show_warn('Database not selected. Please select from dropdown in top of web page')
else:
    try:
        set_logging_enabled(False)

        conn = s2.connect()
        with conn.cursor() as cursor:
            cursor.execute(get_sql_statement())
            results = cursor.fetchall()
            if results is None:
                logging.error('Restore execution failed')
                show_error('Restore Failed')
            else:
                logging.info("Restore completed")
                show_success('Restore completed')
    except s2.exceptions.OperationalError as ope:
        if 'NoSuchBucket' in ope.errmsg:
            logging.error('Provided S3 Bucket does not exists. please check')
            show_error('Provided S3 Bucket does not exists. please check')
        elif 'Access denied' in ope.errmsg:
            logging.error('Failed to restore due to grants missing or firewall settings.please check')
            show_error('Failed to restore due to grants missing or firewall settings.please check')
        else:
            logging.error(f'Failed. error msg: {ope.errmsg}')
            show_error('Restore Failed. error msg: {ope.errmsg}')
    except s2.Error as e:
        logging.error(f'Encountered exception {e}')
        show_error(f'Restore Failed. {str(e)}')

print('\n\nScript execution completed')

### Verify Result

In [8]:
%sql select schema_name from information_schema.SCHEMATA;

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>